In [6]:
import pickle
import pandas
import numpy as np

import keras
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape,LSTM
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Merge 
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from keras import regularizers
from keras import backend as K




In [7]:
#X = np.loadtxt("train_X.csv", delimiter = ",")
X = np.load("train_X_lstm_normalized.pkl.npy")
Y = np.load("train_Y_lstm_normalized.pkl.npy")

In [8]:
#X = X[1150003/2:]

In [9]:
activ_func = 'tanh'
dropout = 0.5
model = Sequential()
model.add(LSTM(512, return_sequences = True, input_shape=(X.shape[1],X.shape[2]), activation=activ_func))
# model.add(Dropout(dropout))
# model.add(LSTM(512, return_sequences = True, activation=activ_func))
# model.add(Dropout(dropout))
# model.add(LSTM(512, activation=activ_func))
model.add(Dropout(dropout))

model.add(Dense(1, kernel_regularizer=regularizers.l2(10))) 
Adm = keras.optimizers.Adam(lr = 0.001, decay = 0.0001)

model.compile(loss='mean_squared_error', optimizer=Adm, metrics=['mae'])
model.summary()


K.tensorflow_backend._get_available_gpus()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 60, 512)           1064960   
_________________________________________________________________
dropout_4 (Dropout)          (None, 60, 512)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 60, 512)           2099200   
_________________________________________________________________
dropout_5 (Dropout)          (None, 60, 512)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total para

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

In [10]:
res = model.fit(X, Y ,batch_size=64, epochs=40,verbose=1, shuffle=True,validation_split = 0.2)

Train on 920002 samples, validate on 230001 samples
Epoch 1/40
920002/920002 [==============================] - 5951s 6ms/step - loss: 0.0914 - mean_absolute_error: 0.0899 - val_loss: 0.8324 - val_mean_absolute_error: 0.8584
Epoch 2/40
920002/920002 [==============================] - 5956s 6ms/step - loss: 0.0465 - mean_absolute_error: 0.0870 - val_loss: 0.8664 - val_mean_absolute_error: 0.8790
Epoch 3/40
920002/920002 [==============================] - 5936s 6ms/step - loss: 0.0463 - mean_absolute_error: 0.0859 - val_loss: 0.8598 - val_mean_absolute_error: 0.8751
Epoch 4/40
 94080/920002 [==>...........................] - ETA: 1:23:01 - loss: 0.0463 - mean_absolute_error: 0.0855

KeyboardInterrupt: 